In [1]:
import requests
import pandas as pd
from tqdm.auto import tqdm
import re
import pickle
from bs4 import BeautifulSoup

In [2]:
df = pd.read_excel("data/SA2_2021_AUST.xlsx")
df.head()

,SA2_CODE_2021,SA2_NAME_2021,CHANGE_FLAG_2021,CHANGE_LABEL_2021,SA3_CODE_2021,SA3_NAME_2021,SA4_CODE_2021,SA4_NAME_2021,GCCSA_CODE_2021,GCCSA_NAME_2021,STATE_CODE_2021,STATE_NAME_2021,AUS_CODE_2021,AUS_NAME_2021,AREA_ALBERS_SQKM,ASGS_LOCI_URI_2021
0,101021007,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...
1,101021008,Karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/SA2/...
2,101021009,Queanbeyan,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,4.7620,http://linked.data.gov.au/dataset/asgsed3/SA2/...
3,101021010,Queanbeyan - East,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.0032,http://linked.data.gov.au/dataset/asgsed3/SA2/...
4,101021012,Queanbeyan West - Jerrabomberra,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.6748,http://linked.data.gov.au/dataset/asgsed3/SA2/...


- SA2_NAME_2021: suburb name
- SA3_NAME_2021: council name
- SA4_NAME_2021: city name
- GCCSA_NAME_2021: region name
- STATE_NAME_2021: state name

# 1. Clean list of suburbs

Things to consider:

| Consideration                              | Raw example           | Clean example                 |
|--------------------------------------------|-----------------------|-------------------------------|
| Split names with '-' into separate suburbs | Emerald - Cockatoo    | [..., Emerald, Cockatoo, ...] |
| Remove any brackets                        | Blacktown (East)      | [..., Blacktown, ...]         |
| Remove cardinal directions                 | Pakenham - North East | [..., Pakenham, ...]          |
| Apply lowercase for website request        | Oakleigh              | [..., oakleigh, ...]          |
| Replace spaces in names with '+'           | Koo Wee Rup           | [..., koo+wee+rup, ...]       |

In [3]:
df[["SA2_NAME_2021", "SA3_NAME_2021", "SA4_NAME_2021", "GCCSA_NAME_2021", "STATE_NAME_2021"]]

,SA2_NAME_2021,SA3_NAME_2021,SA4_NAME_2021,GCCSA_NAME_2021,STATE_NAME_2021
0,Braidwood,Queanbeyan,Capital Region,Rest of NSW,New South Wales
1,Karabar,Queanbeyan,Capital Region,Rest of NSW,New South Wales
2,Queanbeyan,Queanbeyan,Capital Region,Rest of NSW,New South Wales
3,Queanbeyan - East,Queanbeyan,Capital Region,Rest of NSW,New South Wales
4,Queanbeyan West - Jerrabomberra,Queanbeyan,Capital Region,Rest of NSW,New South Wales
...,...,...,...,...,...
2468,Jervis Bay,Jervis Bay,Other Territories,Other Territories,Other Territories
2469,Norfolk Island,Norfolk Island,Other Territories,Other Territories,Other Territories
2470,Migratory - Offshore - Shipping (OT),Migratory - Offshore - Shipping (OT),Migratory - Offshore - Shipping (OT),Migratory - Offshore - Shipping (OT),Other Territories
2471,No usual address (OT),No usual address (OT),No usual address (OT),No usual address (OT),Other Territories


In [4]:
cardinal_directions = [
    "Central",
    "North",
    "North East",
    "East",
    "South East",
    "South",
    "South West",
    "West",
    "North West"
]

suburbs = []

# filter suburbs
for x in df["SA2_NAME_2021"]:
    # if parentheses exist - delete parantheses and whatever is in them 
    x_ = re.sub(r"\([^()]*\)", "", x)
    
    # then split by "-"
    subs = x_.split(" - ")
    
    for sub in subs:
        suburbs.append(sub)

print(len(suburbs))
# remove cardinal names from list of suburbs
suburbs = [x for x in suburbs if x not in cardinal_directions]
print(len(suburbs))

3643
3332


In [5]:
# list of suburbs and possible cardinal combinations
suburbs_dir = suburbs + [f"{direction} {sub}" for sub in tqdm(suburbs) for direction in cardinal_directions]

# convert all to lower case and replace spaces with "+"
suburbs_dir = [x.lower().replace(" ", "+") for x in suburbs_dir]

  0%|          | 0/3332 [00:00<?, ?it/s]

# 2. Scrape data

## 2.1 Scrape LLM friendly data

In [7]:
def scrape_jina_ai(suburb):
    response = requests.get(
        "https://r.jina.ai/" + f"https://www.realestateinvestar.com.au/property/{suburb}",
        headers=headers
    )
    return response.text

In [8]:
suburb_scrape = {}

for suburb in tqdm(suburbs_dir):
    suburb_scrape[suburb] = scrape_jina_ai(suburb)        

  0%|          | 0/33320 [00:00<?, ?it/s]

ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [9]:
# with open("investar_suburbs1.pickle", "wb") as f:
#     pickle.dump(suburb_scrape, f)

## 2.2 Scrape html data

In [135]:
def scrape_tables(soup):
    # get all tables from soup
    tables = soup.find_all('table')
    
    # set table names
    table_names = ["median", "rental", "sales"]
    
    # instantiate empty dict
    dfs = {}
    
    # iterate over the three tables and populate
    for i, table in enumerate(tables):
        # Extract headers (th elements)
        headers = [header.get_text(strip=True) for header in table.find_all('td', class_='datatitle')]

        # Extract rows
        rows = []
        for row in table.find_all('tr')[1:]:  # Skip the header row
            cells = row.find_all('td')
            row_data = [cell.get_text(strip=True) for cell in cells]
            rows.append(row_data)

        # Create a DataFrame
        df = pd.DataFrame(rows, columns=['Metric'] + headers)
        dfs[table_names[i]] = df
    return dfs

def scrape_other_data(soup):
    other_data = {}

    paras = soup.find_all('p')

    vac_rates = []
    rental_stocks = []
    pops = []
    rental_pops = []

    for i, p in enumerate(paras):
        # get region
        if "  Suburb" in p.get_text():
            suburb_name = p.get_text().split("\n")[2]
            other_data["suburb"] = {"name": suburb_name}

        # get region
        if "  Region" in p.get_text():
            region_name = p.get_text().split("\n")[2]
            other_data["region"] = {"name": region_name}

        # get state
        if "  State" in p.get_text():
            #print(p.get_text())
            state_name = p.get_text().split("\n")[2]
            other_data["state"] = {"name": state_name}

        # get vac rate
        if "Current vacancy rate" in p.get_text():
            vac_rates.append(paras[i-1].get_text())

        # get rental stock
        if "Rental stock available" in p.get_text():
            rental_stocks.append(paras[i-1].get_text())

        # get pop
        if "Population" in p.get_text():
            pops.append(paras[i-1].get_text())

        # get rental pop
        if "Rental population" in p.get_text():
            rental_pops.append(paras[i-1].get_text())

    if suburb == "act":
        sas = ["state"]
    else:
        sas = ["suburb", "region", "state"]
    for (sa, vr, rs, p, rp) in zip(sas, vac_rates, rental_stocks, pops, rental_pops):
        other_data[sa]["vacancy_rate"] = vr
        other_data[sa]["rental_stock"] = rs
        other_data[sa]["population"] = p
        other_data[sa]["rental_pop"] = rp
        
    return other_data

In [18]:
suburb_scrape = {}

# scrape tabular data
for suburb in tqdm(suburbs_dir):
    # get html
    page = requests.get(f'https://www.realestateinvestar.com.au/property/{suburb}')
    
    if page.status_code == 200:
        # convert to soup
        soup = BeautifulSoup(page.text, 'html.parser')
        
        suburb_scrape[suburb] = scrape_tables(soup)
        
    else:
        pass
    
with open("investar_tables.pickle", "wb") as f:
    pickle.dump(suburb_scrape, f)

  0%|          | 0/33320 [00:00<?, ?it/s]

In [32]:
valid_suburbs = list(suburb_scrape.keys())

In [78]:
# scrape other data
suburb_soups = {}

for suburb in tqdm(valid_suburbs):
    # get html
    page = requests.get(f'https://www.realestateinvestar.com.au/property/{suburb}')
    
    # save soup
    soup = BeautifulSoup(page.text, 'html.parser')
    suburb_soups[suburb] = soup
    
# with open("investar_soups.pickle", "wb") as f:
#     pickle.dump(suburb_soups, f)

  0%|          | 0/2774 [00:00<?, ?it/s]

RecursionError: maximum recursion depth exceeded while getting the str of an object

In [138]:
suburb_other_data = {}
for suburb, soup in tqdm(suburb_soups.items()):
    suburb_other_data[suburb] = pd.DataFrame(scrape_other_data(soup))

  0%|          | 0/2774 [00:00<?, ?it/s]

In [139]:
with open("other_data.pickle", "wb") as f:
    pickle.dump(suburb_other_data, f)

In [31]:
# # save list of valid suburbs
# with open("data/suburbs.pickle", "wb") as f:
#     pickle.dump(list(suburb_scrape.keys()), f)

# 3. Clean scraped data

In [2]:
# with open("investar_suburbs.pickle", "rb") as f:
#     raw = pickle.load(f)